In [ ]:
import random
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import nltk
from nltk.tokenize import word_tokenize
from collections import defaultdict
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
# CUDA 설정
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# NLTK 다운로드
nltk.download('punkt')

# 데이터 샘플
data = [
    ("hello", "안녕"),
    ("python", "파이썬"),
    ("lol", "리그오브레전드"),
    ("java", "자바"),
    ("apple", "사과"),
    ("dl", "딥러닝"),
    ("ml", "머신러닝"),
    ("computer", "컴퓨터"),
    ("programming", "프로그래밍"),
    ("language", "언어"),
    ("artificial", "인공적인"),
    ("intelligence", "지능"),
    ("neural", "신경"),
    ("network", "네트워크"),
    ("data", "데이터"),
    ("science", "과학"),
    ("algorithm", "알고리즘"),
    ("model", "모델"),
    ("training", "훈련"),
    ("learning", "학습"),
    ("hello", "안녕"),
    ("python", "파이썬"),
    ("lol", "리그오브레전드"),
    ("java", "자바"),
    ("apple", "사과"),
    ("dl", "딥러닝"),
    ("ml", "머신러닝"),
    ("computer", "컴퓨터"),
    ("programming", "프로그래밍"),
    ("language", "언어"),
    ("artificial", "인공적인"),
    ("intelligence", "지능"),
    ("neural", "신경"),
    ("network", "네트워크"),
    ("data", "데이터"),
    ("science", "과학"),
    ("algorithm", "알고리즘"),
    ("model", "모델"),
    ("training", "훈련"),
    ("hello", "안녕"),
    ("python", "파이썬"),
    ("lol", "리그오브레전드"),
    ("java", "자바"),
    ("apple", "사과"),
    ("dl", "딥러닝"),
    ("ml", "머신러닝"),
    ("computer", "컴퓨터"),
    ("programming", "프로그래밍"),
    ("language", "언어"),
    ("artificial", "인공적인"),
    ("intelligence", "지능"),
    ("neural", "신경"),
    ("network", "네트워크"),
    ("data", "데이터"),
    ("science", "과학"),
    ("algorithm", "알고리즘"),
    ("model", "모델"),
    ("training", "훈련"),
    ("hello", "안녕"),
    ("python", "파이썬"),
    ("lol", "리그오브레전드"),
    ("java", "자바"),
    ("apple", "사과"),
    ("dl", "딥러닝"),
    ("ml", "머신러닝"),
    ("computer", "컴퓨터"),
    ("programming", "프로그래밍"),
    ("language", "언어"),
    ("artificial", "인공적인"),
    ("intelligence", "지능"),
    ("neural", "신경"),
    ("network", "네트워크"),
    ("data", "데이터"),
    ("science", "과학"),
    ("algorithm", "알고리즘"),
    ("model", "모델"),
    ("training", "훈련"),
    ("hello", "안녕"),
    ("python", "파이썬"),
    ("lol", "리그오브레전드"),
    ("java", "자바"),
    ("apple", "사과"),
    ("dl", "딥러닝"),
    ("ml", "머신러닝"),
    ("computer", "컴퓨터"),
    ("programming", "프로그래밍"),
    ("language", "언어"),
    ("artificial", "인공적인"),
    ("intelligence", "지능"),
    ("neural", "신경"),
    ("network", "네트워크"),
    ("data", "데이터"),
    ("science", "과학"),
    ("algorithm", "알고리즘"),
    ("model", "모델"),
    ("training", "훈련"),
    ("hello", "안녕"),
    ("python", "파이썬"),
    ("lol", "리그오브레전드"),
    ("java", "자바"),
    ("apple", "사과"),
    ("dl", "딥러닝"),
    ("ml", "머신러닝"),
    ("computer", "컴퓨터"),
    ("programming", "프로그래밍"),
    ("language", "언어"),
    ("artificial", "인공적인"),
    ("intelligence", "지능"),
    ("neural", "신경"),
    ("network", "네트워크"),
    ("data", "데이터"),
    ("science", "과학"),
    ("algorithm", "알고리즘"),
    ("model", "모델"),
    ("hello", "안녕"),
    ("python", "파이썬"),
    ("lol", "리그오브레전드"),
    ("java", "자바"),
    ("apple", "사과"),
    ("dl", "딥러닝"),
    ("ml", "머신러닝"),
    ("computer", "컴퓨터"),
    ("programming", "프로그래밍"),
    ("language", "언어"),
    ("artificial", "인공적인"),
    ("intelligence", "지능"),
    ("neural", "신경"),
    ("network", "네트워크"),
    ("data", "데이터"),
    ("science", "과학"),
    ("algorithm", "알고리즘"),
    ("model", "모델"),
    ("hello", "안녕"),
    ("python", "파이썬"),
    ("lol", "리그오브레전드"),
    ("java", "자바"),
    ("apple", "사과"),
    ("dl", "딥러닝"),
    ("ml", "머신러닝"),
    ("computer", "컴퓨터"),
    ("programming", "프로그래밍"),
    ("language", "언어"),
    ("artificial", "인공적인"),
    ("intelligence", "지능"),
    ("neural", "신경"),
    ("network", "네트워크"),
    ("data", "데이터"),
    ("science", "과학"),
    ("algorithm", "알고리즘"),
    ("model", "모델"),
    ("hello", "안녕"),
    ("python", "파이썬"),
    ("lol", "리그오브레전드"),
    ("java", "자바"),
    ("apple", "사과"),
    ("dl", "딥러닝"),
    ("ml", "머신러닝"),
    ("computer", "컴퓨터"),
    ("programming", "프로그래밍"),
    ("language", "언어"),
    ("artificial", "인공적인"),
    ("intelligence", "지능"),
    ("neural", "신경"),
    ("network", "네트워크"),
    ("data", "데이터"),
    ("science", "과학"),
    ("algorithm", "알고리즘"),
    ("model", "모델"),
]


class TranslationData(Dataset):
    def __init__(self, data, max_length=100):
        self.eng_sentence = [word[0] for word in data]
        self.kor_sentence = [word[1] for word in data]
        self.eng_vocab = self.build_vocab(self.eng_sentence)
        self.kor_vocab = self.build_vocab(self.kor_sentence)

        self.sos_idx = self.kor_vocab['<sos>']
        self.eos_idx = self.kor_vocab['<eos>']
        self.padding_idx = self.kor_vocab['<pad>']

        self.max_length = max_length

        print("영어 문장:", self.eng_sentence)
        print("한국어 문장:", self.kor_sentence)
        print("영어 어휘:", self.eng_vocab)
        print("한국어 어휘:", self.kor_vocab)

    def build_vocab(self, sentences):
        vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}
        for sentence in sentences:
            for word in word_tokenize(sentence):
                if word not in vocab:
                    vocab[word] = len(vocab)
        return vocab

    def __len__(self):
        return len(self.kor_sentence)

    def __getitem__(self, idx):
        eng = self.eng_sentence[idx]
        kor = self.kor_sentence[idx]


        eng_indices = [self.eng_vocab[word] for word in word_tokenize(eng) if word in self.eng_vocab]

        kor_indices = [self.sos_idx] + [self.kor_vocab[word] for word in word_tokenize(kor) if word in self.kor_vocab] + [self.eos_idx]
        # 패딩 추가
        if len(eng_indices) < self.max_length:
          eng_indices += [self.padding_idx] * (self.max_length - len(eng_indices))
        else:
          eng_indices = eng_indices[:self.max_length]  # max_length 초과시 잘라냄

        if len(kor_indices) < self.max_length:
          kor_indices += [self.padding_idx] * (self.max_length - len(kor_indices))
        else:
          kor_indices = kor_indices[:self.max_length]

        return torch.tensor(eng_indices), torch.tensor(kor_indices)

dataset = TranslationData(data)

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.Tanh()
        torch.nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, input_seq):
        batch_size, seq_length = input_seq.size()
        hidden = torch.zeros(batch_size, self.hidden_size).to(input_seq.device)

        for char_idx in range(seq_length):
            x_t = self.embedding(input_seq[:, char_idx])
            hidden = self.activation(self.linear(x_t) + hidden)

        return hidden

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.Tanh()
        self.linear2 = nn.Linear(hidden_size, output_size)
        torch.nn.init.xavier_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)

    def forward(self, input, hidden):
        embedded = self.embedding(input)

        batch_size = embedded.size(0)
        seq_len = embedded.size(1)

        outputs = torch.zeros(batch_size, seq_len, self.output_size).to(input.device)

        hidden = hidden[-1]

        for char_idx in range(seq_len):
            if char_idx == 0:
                previous_y = torch.zeros(batch_size, self.hidden_size).to(input.device)
            else:
                y_prev = input[:, char_idx - 1]
                previous_y = self.embedding(y_prev).view(batch_size, -1)

            hidden = self.activation(self.linear1(previous_y) + hidden)
            output = self.linear2(hidden)

            outputs[:, char_idx, :] = output
        return outputs, hidden




class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.output_size = decoder.output_size

    def forward(self, input_seq, decoder_target, teacher_forcing_ratio=0.1):
        encoder_hidden = self.encoder(input_seq)
        input_seq = input_seq.to(device)
        decoder_input = decoder_target[0].unsqueeze(0).to(device)
        outputs = []

        for t in range(decoder_target.size(0)):
            if t == 0:
                decoder_output, encoder_hidden = self.decoder(decoder_input, encoder_hidden)
            else:
                decoder_output, encoder_hidden = self.decoder(decoder_input, encoder_hidden)

            decoder_output = decoder_output.view(1, -1, self.output_size)
            outputs.append(decoder_output)

            if t < decoder_target.size(0) - 1:
                top1 = decoder_output.argmax(2)
                decoder_input = decoder_target[t + 1].unsqueeze(0).to(device) if random.random() < teacher_forcing_ratio else top1.to(device)
            else:
                decoder_input = decoder_output

        outputs = torch.cat(outputs, dim=0)
        return outputs

def translate(model, word, dataset, device):
    if word not in dataset.eng_vocab:
        return "데이터에 해당 단어가 없습니다."

    eng_indices = [dataset.eng_vocab[word]]
    eng_indices += [dataset.padding_idx] * (dataset.max_length - len(eng_indices))
    test_input = torch.tensor(eng_indices).unsqueeze(0).to(device)

    with torch.no_grad():
        test_target = torch.zeros((1, dataset.max_length), dtype=torch.long).to(device)
        test_target[0][0] = dataset.kor_vocab['<sos>']
        output = model(test_input, test_target)

        predicted = torch.argmax(output, dim=2)
        for pred in predicted[0]:
            if pred.item() not in {dataset.padding_idx, dataset.sos_idx, dataset.eos_idx}:
                translated_word = list(dataset.kor_vocab.keys())[list(dataset.kor_vocab.values()).index(pred.item())]
                return translated_word

        # return " ".join(translated_sentence)


def train_model(model, dataloader, criterion, optimizer, num_epochs, device):
    model.train()  
    loss_values = []  
    
    for epoch in range(num_epochs):
        epoch_loss = 0  
        for eng, kor in dataloader:
            eng, kor = eng.to(device), kor.to(device)

            optimizer.zero_grad() 
            output = model(eng, kor[:, :-1])  
            output = output.view(-1, output.size(-1)) 

            # 손실 계산
            loss = criterion(output, kor[:, 1:].contiguous().view(-1))

            if torch.isnan(loss):
                print("손실이 NaN입니다.")
                continue

            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트
            epoch_loss += loss.item()  # 에포크 손실 누적
        
        avg_loss = epoch_loss / len(dataloader)  # 평균 손실 계산
        loss_values.append(avg_loss)  # 리스트에 평균 손실 추가

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')
    
    return loss_values  # 손실 값을 반환

# 손실 값 리스트를 받는 코드
loss_values = train_model(model, dataloader, criterion, optimizer, num_epochs, device)

# 손실 값 그래프 그리기
plt.plot(loss_values)
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.show() 



def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for eng, kor in dataloader:
            eng, kor = eng.to(device), kor.to(device)
            output = model(eng, kor[:, :-1])
            predicted = torch.argmax(output, dim=2)

            for i in range(predicted.size(0)):
                for j in range(predicted.size(1)):
                    if kor[i, j + 1] != dataset.padding_idx:
                        total += 1
                        correct += (predicted[i, j] == kor[i, j + 1]).item()

    return correct / total if total > 0 else 0.0



if __name__ == '__main__':
    max_length = 10
    hidden_size = 100
    batch_size = 100
    num_epochs = 10
    learning_rate = 0.001


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # device = torch.device("cpu")


    dataset = TranslationData(data, max_length)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    print(device)


    eng_vocab_size = len(dataset.eng_vocab)
    kor_vocab_size = len(dataset.kor_vocab)
    # print("Encoder input size:", eng_vocab_size)
    # print("Decoder input size:", kor_vocab_size)
    # print("Decoder output size:", kor_vocab_size)
    # print("Input sequence device:", input_seq.device)
    # print("Target sequence device:", input.device)


    try:
        encoder = Encoder(input_size=eng_vocab_size, hidden_size=hidden_size).to(device)
        decoder = Decoder(input_size=kor_vocab_size, hidden_size=hidden_size, output_size=kor_vocab_size).to(device)
    except Exception as e:
        print(f"모델 초기화 중 오류 발생: {e}")

    model = Seq2Seq(encoder, decoder).to(device)
    for input_seq, target_seq in dataloader:
      input_seq = input_seq.to(device)
      target_seq = target_seq.to(device)
      output = model(input_seq, target_seq)
      # print("모델 출력:", output)
      print("Seq2Seq output shape:", output.shape)  # Seq2Seq 출력 모양 출력
      break


    criterion = nn.CrossEntropyLoss(ignore_index=dataset.padding_idx)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)


    train_model(model, dataloader, criterion, optimizer, num_epochs, device)


    acc = evaluate_model(model, dataloader, device)
    print(f"Training Accuracy: {acc * 100:.2f}%\n")
    

    while True:
        word = input("영단어 입력: ")
        if word.lower() == 'exit':
            break
        result = translate(model, word, dataset, device)
        print("번역 결과:", result)
    
